## Import the required packages

In [1]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
# keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Lambda
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.optimizers import Adam
from keras.models import load_model
import glob
import matplotlib.pyplot as plt
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.feature import hog
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# from sklearn.svm import LinearSVC
from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import tensorflow as tf
from tensorflow.python.framework import ops
import time
import h5py
%matplotlib inline

/Users/manika/anaconda3/envs/tf14/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/manika/anaconda3/envs/tf14/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Loading the vehicle and non-vehicle data

In [2]:
car = glob.glob('vehicles/**/*.png')
noncar = glob.glob('non-vehicles/**/*.png')
print ("No. of car images: ", len(car))
print ("No. of non-car images: ", len(noncar))

No. of car images:  8792
No. of non-car images:  8968


In [3]:
cars = []
for image in car:
    cars.append(cv2.imread(image))
non_cars = []
for image in noncar:
    non_cars.append(cv2.imread(image))

In [4]:
print (len(cars))
print (len(non_cars))

8792
8968


In [5]:
# X = np.asarray(cars[0:7000] + non_cars[0:7000])
# y = np.asarray([1]*len(cars[0:7000]) + [0]*len(non_cars[0:7000]))
X = np.asarray(cars + non_cars)
y = np.asarray([1]*len(cars) + [0]*len(non_cars))

In [6]:
# X_norm = (X/255.0)

In [7]:
def one_hot_encoding(y, numOfClasses):
    """
    Converts label into one-hot encoding.
    takes in the labels and the number of classes 
    """
    encoded = np.zeros((y.shape[0],numOfClasses))
    for i in range(y.shape[0]):
        encoded[i][y[i]] = 1.0
    return encoded

y_encoded = one_hot_encoding(y, 2)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [9]:
def get_conv(input_shape=(64, 64, 3), filename=None):
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=input_shape, output_shape=input_shape))
    model.add(Convolution2D(10, 3, 3, activation='relu', name='conv1', input_shape=input_shape, border_mode="same"))
    model.add(Convolution2D(10, 3, 3, activation='relu', name='conv2', border_mode="same"))
    model.add(MaxPooling2D(pool_size=(8, 8)))
    model.add(Dropout(0.25))
    model.add(Convolution2D(128, 8, 8, activation="relu", name="dense1")) # This was Dense(128)
    model.add(Dropout(0.5))
    model.add(Convolution2D(1, 1, 1, name="dense2", activation="tanh")) # This was Dense(1)
    if filename:
        model.load_weights(filename)        
    return model

model = get_conv()
model.add(Flatten())
model.compile(loss='mse', optimizer='adadelta', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=64, nb_epoch=10, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 14208 samples, validate on 3552 samples
Epoch 1/10
14208/14208 [==============================] - 61s 4ms/step - loss: 0.0772 - acc: 0.9043 - val_loss: 0.0280 - val_acc: 0.9733
Epoch 2/10
14208/14208 [==============================] - 59s 4ms/step - loss: 0.0241 - acc: 0.9721 - val_loss: 0.0154 - val_acc: 0.9817
Epoch 3/10
14208/14208 [==============================] - 62s 4ms/step - loss: 0.0157 - acc: 0.9828 - val_loss: 0.0105 - val_acc: 0.9896
Epoch 4/10
14208/14208 [==============================] - 59s 4ms/step - loss: 0.0122 - acc: 0.9861 - val_loss: 0.0082 - val_acc: 0.9913
Epoch 5/10
14208/14208 [==============================] - 64s 4ms/step - loss: 0.0107 - acc: 0.9875 - val_loss: 0.0081 - val_acc: 0.9896
Epoch 6/10
14208/14208 [==============================] - 64s 4ms/step - loss: 0.0097 - acc: 0.9885 - val_loss: 0.0077 - val_acc: 0.9901
Epoch 7/10
14208/14208 [==============================] - 76s 5ms/step - loss: 0.0084 - acc: 0.9899 - val_loss: 0.0106 - val_acc:

In [10]:
model.save_weights("result_new_.h5")

In [11]:
from scipy.ndimage.measurements import label

model = get_conv(input_shape=(None,None,3), filename="result_new_.h5")

prev_hot_windows1 = []
prev_hot_windows2 = []
prev_hot_windows3 = []


def process2(img):   
    global prev_hot_windows1, prev_hot_windows2, prev_hot_windows3    
    pred = model.predict(img.reshape(1, img.shape[0], img.shape[1], img.shape[2]))

    xx, yy = np.meshgrid(np.arange(pred.shape[2]),np.arange(pred.shape[1]))
    box_x = (xx[pred[0, :, :, 0] > 0.99])
    box_y = (yy[pred[0, :, :, 0] > 0.99])
    
    heat = np.zeros_like(img[:, :, 0]).astype(np.float)

    for x, y in zip(box_x, box_y):
        if (y*8 > 375) & (x*8 > 500):
            x1 = x*8
            x2 = x*8 + 70
            y1 = y*8
            y2 = y*8 + 70
            heat[y1:y2, x1:x2] += 1
            
    for bbox in prev_hot_windows1:
        heat[(bbox[0][1]):(bbox[1][1]), (bbox[0][0]):(bbox[1][0])] += 4
    for bbox in prev_hot_windows2:
        heat[(bbox[0][1]):(bbox[1][1]), (bbox[0][0]):(bbox[1][0])] += 3   
    for bbox in prev_hot_windows3:
        heat[(bbox[0][1]):(bbox[1][1]), (bbox[0][0]):(bbox[1][0])] += 2          
            
    heat[heat <= 9] = 0
    
    prev_hot_windows3 = list(prev_hot_windows2)
    prev_hot_windows2 = list(prev_hot_windows1)
    prev_hot_windows1 = []
    
    labels = label(heat)
    for car_number in range(1, labels[1]+1):
        nonzero = (labels[0] == car_number).nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        w = np.max(nonzerox)-np.min(nonzerox)
        h = np.max(nonzeroy)-np.min(nonzeroy)
        if (50 < w < 400) & (25 < h < 250):
            cv2.rectangle(img, bbox[0], bbox[1], (0, 0, 255), 6) 
            prev_hot_windows1.append(bbox)
            
    return img

In [12]:
from moviepy.editor import VideoFileClip

result_file2 = 'result_video_.mp4'
clip2 = VideoFileClip('test_video.mp4')
clip2_ = clip2.fl_image(process2) 
%time clip2_.write_videofile(result_file2, audio=False)

[MoviePy] >>>> Building video result_video_.mp4
[MoviePy] Writing video result_video_.mp4


100%|█████████▉| 1260/1261 [15:26<00:00,  1.36it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result_video_.mp4 

CPU times: user 28min 54s, sys: 4min 34s, total: 33min 28s
Wall time: 15min 29s
